In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
import requests

### Funções

In [2]:
# Função que busca os dados e transforma em um dataframe
def buscar_dados(url):
    # Pego os dados
    text = requests.get(url)
    data = json.loads(text.content)
    
    # Transformo em df
    df = pd.DataFrame(index=range(0,5000),columns=['variavel', 'codigo', 'nome', 'data', 'valor'])
    cont=0
    for x in range(0, len(data)):
        for y in range(0, len(data[x]['resultados'])):
            df.loc[cont,'variavel'] = data[x]['variavel']
            df.loc[cont,'codigo'] = list(data[x]['resultados'][y]['classificacoes'][0]['categoria'])[0]
            df.loc[cont,'nome'] = list(data[x]['resultados'][y]['classificacoes'][0]['categoria'].values())[0]
            df.loc[cont,'data'] = list(data[x]['resultados'][y]['series'][0]['serie'])[0]
            df.loc[cont,'valor'] = list(data[x]['resultados'][y]['series'][0]['serie'].values())[0]

            cont=cont+1
            
            
    df = df.dropna()
    df.set_index('nome', inplace=True)
    
    number = len(df['variavel'].drop_duplicates())
    lista = list(df['variavel'].drop_duplicates())
    
    for a in range(0, number):
        lista[a] = df[df['variavel']==lista[a]]
        
    df2 = pd.concat(lista, axis=1)['valor']
    df2.columns = list(df['variavel'].drop_duplicates())
    df2 = df2.where((df2!='-')&(df2!='...')&(df2!='..')&(df2!='.'), np.nan).astype('float')
    
    return df2

In [3]:
# Função que transforma o df em agregado (sem subitens)
def agregar(df, identador=1, separador='.'):
    '''
    df = (pandas.DataFrame) dataframe da função buscar_dados
    identador = (int) Qual letra da string procurar
    separador = (string) Qual string está nessa letra
    
    '''  
    
    global agregados
    agregados = []
    for z in range(0, len(df.index)):
        if df.index[z][identador] == separador:
            agregados.append(z)
            
            
    df = df.iloc[agregados]        
            
    return df        

# Análise do IPCA

In [4]:
# Retorna o dataframe dos itens do IPCA
ipca = buscar_dados('https://servicodados.ibge.gov.br/api/v3/agregados/7060/periodos/202102/variaveis/63|2265?localidades=N1[all]&classificacao=315[all]')

In [5]:
ipca

,IPCA - Variação mensal,IPCA - Variação acumulada em 12 meses
nome,,
Índice geral,0.86,5.20
1.Alimentação e bebidas,0.27,15.00
11.Alimentação no domicílio,0.28,19.42
"1101.Cereais, leguminosas e oleaginosas",-1.29,57.83
1101002.Arroz,-1.52,69.80
...,...,...
9101010.Tv por assinatura,0.00,0.00
9101018.Acesso à internet,0.00,8.51
9101019.Aparelho telefônico,-0.70,5.51


In [6]:
ipca['IPCA - Variação mensal'].mean()

0.6639606126914661

In [7]:
ipca['IPCA - Variação mensal'].where(abs(ipca['IPCA - Variação mensal'])<5.304291816820741, np.nan).dropna().mean()

0.509331797235023

### Maiores variações no mês

In [8]:
agregar(ipca, 1, '.')['IPCA - Variação mensal'].sort_values(ascending=False)

nome
8.Educação                     2.48
5.Transportes                  2.28
3.Artigos de residência        0.66
6.Saúde e cuidados pessoais    0.62
2.Habitação                    0.40
4.Vestuário                    0.38
1.Alimentação e bebidas        0.27
7.Despesas pessoais            0.17
9.Comunicação                 -0.13
Name: IPCA - Variação mensal, dtype: float64

### Maiores variações no acumulado de 12 meses

In [9]:
agregar(ipca, 1, '.')['IPCA - Variação acumulada em 12 meses'].sort_values(ascending=False)

nome
1.Alimentação e bebidas        15.00
3.Artigos de residência         7.78
2.Habitação                     4.39
5.Transportes                   3.66
9.Comunicação                   2.98
6.Saúde e cuidados pessoais     2.05
7.Despesas pessoais             0.93
4.Vestuário                     0.39
8.Educação                     -0.09
Name: IPCA - Variação acumulada em 12 meses, dtype: float64

### Dentro de Alimentação, as maiores altas:

In [10]:
maiores = []
for z in range(0, len(ipca.index)):
    if ipca.index[z][0] == ipca.iloc[agregados]['IPCA - Variação acumulada em 12 meses'].sort_values(ascending=False).head(1).index[0][0]:
        maiores.append(z)

In [11]:
ipca.iloc[maiores]['IPCA - Variação acumulada em 12 meses'].sort_values(ascending=False).head(10)

nome
1113013.Óleo de soja                       87.59
1106015.Limão                              79.01
1101002.Arroz                              69.80
1103043.Cebola                             69.45
1103020.Abobrinha                          63.71
1103002.Batata-doce                        62.28
1101053.Feijão - macáçar (fradinho)        59.52
1101.Cereais, leguminosas e oleaginosas    57.83
1113.Óleos e gorduras                      55.98
1101052.Feijão - preto                     51.60
Name: IPCA - Variação acumulada em 12 meses, dtype: float64

# Análise PIMPF

### Produção Física Industrial, por grandes categorias econômicas

In [12]:
pfi = buscar_dados('https://servicodados.ibge.gov.br/api/v3/agregados/3651/periodos/202101/variaveis/3135|3134|3136|3137|3138|4139|3139|3140|3141?localidades=N1[all]&classificacao=543[all]')

In [13]:
pfi

,Índice de base fixa sem ajuste sazonal (Base: média de 2012 = 100),Índice de base fixa com ajuste sazonal (Base: média de 2012 = 100),Índice mensal (Base: igual mês do ano anterior = 100),Índice acumulado no ano (Base: igual período do ano anterior = 100),Índice acumulado nos últimos 12 meses (Base: últimos 12 meses anteriores = 100),Variação percentual mês/mês imediatamente anterior com ajuste sazonal (Base: mês imediatamente anterior),Variação percentual mensal (Base: igual mês do ano anterior),Variação percentual acumulada no ano (Base: igual período do ano anterior),Variação percentual acumulada nos últimos 12 meses (Base: últimos 12 meses anteriores)
nome,,,,,,,,,
1 Bens de capital,77.4,93.9,117.2,117.2,91.1,4.6,17.2,17.2,-8.9
"110 Bens de capital, exceto equipamentos de transporte industrial",80.6,NaN,114.6,114.6,96.9,NaN,14.6,14.6,-3.1
120 Equipamentos de transporte industrial,71.7,NaN,122.8,122.8,80.5,NaN,22.8,22.8,-19.5
2 Bens intermediários,83.8,90.5,102.8,102.8,99.3,-1.0,2.8,2.8,-0.7
"210 Alimentos e bebidas básicos, destinados principalmente à indústria",88.1,NaN,93.4,93.4,101.9,NaN,-6.6,-6.6,1.9
"220 Alimentos e bebidas elaborados, destinados principalmente à indústria",52.5,NaN,93.1,93.1,112.3,NaN,-6.9,-6.9,12.3
230 Insumos industriais básicos,70.4,NaN,110.1,110.1,94.9,NaN,10.1,10.1,-5.1
240 Insumos industriais elaborados,89.3,NaN,107.4,107.4,99.0,NaN,7.4,7.4,-1.0
250 Combustíveis e lubrificantes básicos,116.2,NaN,90.7,90.7,102.0,NaN,-9.3,-9.3,2.0


In [14]:
agregar(pfi,1,' ')

,Índice de base fixa sem ajuste sazonal (Base: média de 2012 = 100),Índice de base fixa com ajuste sazonal (Base: média de 2012 = 100),Índice mensal (Base: igual mês do ano anterior = 100),Índice acumulado no ano (Base: igual período do ano anterior = 100),Índice acumulado nos últimos 12 meses (Base: últimos 12 meses anteriores = 100),Variação percentual mês/mês imediatamente anterior com ajuste sazonal (Base: mês imediatamente anterior),Variação percentual mensal (Base: igual mês do ano anterior),Variação percentual acumulada no ano (Base: igual período do ano anterior),Variação percentual acumulada nos últimos 12 meses (Base: últimos 12 meses anteriores)
nome,,,,,,,,,
1 Bens de capital,77.4,93.9,117.2,117.2,91.1,4.6,17.2,17.2,-8.9
2 Bens intermediários,83.8,90.5,102.8,102.8,99.3,-1.0,2.8,2.8,-0.7
3 Bens de consumo,80.0,92.8,98.7,98.7,91.0,0.8,-1.3,-1.3,-9.0
9 Bens não especificados anteriormente,64.0,NaN,86.6,86.6,80.9,NaN,-13.4,-13.4,-19.1


### Produção Física Industrial, por seções e atividades industriais

In [15]:
pfis = buscar_dados('https://servicodados.ibge.gov.br/api/v3/agregados/3653/periodos/202101/variaveis/3135|3134|3136|3137|3138|4139|3139|3140|3141?localidades=N1[all]&classificacao=544[all]')

In [16]:
pfis

,Índice de base fixa sem ajuste sazonal (Base: média de 2012 = 100),Índice de base fixa com ajuste sazonal (Base: média de 2012 = 100),Índice mensal (Base: igual mês do ano anterior = 100),Índice acumulado no ano (Base: igual período do ano anterior = 100),Índice acumulado nos últimos 12 meses (Base: últimos 12 meses anteriores = 100),Variação percentual mês/mês imediatamente anterior com ajuste sazonal (Base: mês imediatamente anterior),Variação percentual mensal (Base: igual mês do ano anterior),Variação percentual acumulada no ano (Base: igual período do ano anterior),Variação percentual acumulada nos últimos 12 meses (Base: últimos 12 meses anteriores)
nome,,,,,,,,,
1 Indústria geral,81.8,91.4,102.3,102.3,95.8,0.4,2.3,2.3,-4.2
2 Indústrias extrativas,88.0,87.7,100.3,100.3,98.0,1.0,0.3,0.3,-2.0
3 Indústrias de transformação,81.1,92.5,102.6,102.6,95.5,0.1,2.6,2.6,-4.5
3.10 Fabricação de produtos alimentícios,75.0,96.0,94.3,94.3,103.8,2.9,-5.7,-5.7,3.8
3.11 Fabricação de bebidas,104.6,99.8,99.5,99.5,99.6,-0.3,-0.5,-0.5,-0.4
3.12 Fabricação de produtos do fumo,45.2,80.9,102.5,102.5,109.9,-10.8,2.5,2.5,9.9
3.13 Fabricação de produtos têxteis,87.4,98.3,120.6,120.6,94.7,-3.2,20.6,20.6,-5.3
3.14 Confecção de artigos do vestuário e acessórios,67.2,87.9,106.6,106.6,76.7,0.5,6.6,6.6,-23.3
"3.15 Preparação de couros e fabricação de artefatos de couro, artigos para viagem e calçados",81.7,94.7,106.2,106.2,81.6,1.9,6.2,6.2,-18.4


### Produção física industrial por grupos e classes industriais selecionados

In [17]:
pfigce = buscar_dados('https://servicodados.ibge.gov.br/api/v3/agregados/3650/periodos/202101/variaveis/3135|3136|3137|3138|3139|3140|3141?localidades=N1[all]&classificacao=542[all]')

In [18]:
pfigce

,Índice de base fixa sem ajuste sazonal (Base: média de 2012 = 100),Índice mensal (Base: igual mês do ano anterior = 100),Índice acumulado no ano (Base: igual período do ano anterior = 100),Índice acumulado nos últimos 12 meses (Base: últimos 12 meses anteriores = 100),Variação percentual mensal (Base: igual mês do ano anterior),Variação percentual acumulada no ano (Base: igual período do ano anterior),Variação percentual acumulada nos últimos 12 meses (Base: últimos 12 meses anteriores)
nome,,,,,,,
10.1 Abate e fabricação de produtos de carne,98.1,96.6,96.6,99.2,-3.4,-3.4,-0.8
"10.11 Abate de reses, exceto suínos",92.8,93.9,93.9,95.0,-6.1,-6.1,-5.0
"10.12 Abate de suínos, aves e outros pequenos animais",101.0,98.0,98.0,102.7,-2.0,-2.0,2.7
10.13 Fabricação de produtos de carne,110.2,100.8,100.8,97.0,0.8,0.8,-3.0
"10.3 Fabricação de conservas de frutas, legumes e outros vegetais",66.5,94.3,94.3,73.8,-5.7,-5.7,-26.2
...,...,...,...,...,...,...,...
"32.1 Fabricação de artigos de joalheria, bijuteria e semelhantes",11.4,47.0,47.0,27.6,-53.0,-53.0,-72.4
32.3 Fabricação de artefatos para pesca e esporte,140.9,159.0,159.0,135.6,59.0,59.0,35.6
32.4 Fabricação de brinquedos e jogos recreativos,43.6,144.3,144.3,98.2,44.3,44.3,-1.8


In [19]:
agregados = []
for z in range(0, len(pfigce.index)):
    if pfigce.index[z][3] == '1' and pfigce.index[z][4] == ' ':
        agregados.append(z)